# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy


In [2]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
#Exclusao dos dados nulos
df1 = df.dropna()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes:

In [5]:
#Exclusao das variaveis irrelevantes a analise
df1 = df1.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])
df1.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [6]:
print(df1.columns)

Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda'],
      dtype='object')


In [7]:
#1. Separaçao de base em treinamento e teste (25% teste, 75% treinamento)
X = df1.drop(columns=['renda'])
X = pd.get_dummies(X, drop_first=True)
X[['posse_de_veiculo', 'posse_de_imovel']] = X[['posse_de_veiculo','posse_de_imovel']].astype(int)

y = df1['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [8]:
#2. Regularização ridge com alpha = [0]
alpha = 0
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2}")

NameError: name 'r2' is not defined

In [ ]:
# Regularização ridge com alpha = [0.001]
alpha = 0.001
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2}")

In [ ]:
# Regularização ridge com alpha = [0.005]
alpha = 0.005
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2}")

In [ ]:
# Regularização ridge com alpha = [0.01]
alpha = 0.01
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2}")

In [ ]:
# Regularização ridge com alpha = [0.05]
alpha = 0.05
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2}")

In [ ]:
# Regularização ridge com alpha = [0.1]
alpha = 0.1
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2}")

**Resultados de regularização Ridge**:

- Alpha: 0 - R²: 0.2925875510859858
- Alpha: 0.001 - R²: 0.29258756799265095
- Alpha: 0.005 - R²: 0.29258763551736733
- Alpha: 0.01 - R²: 0.2925877196947876
- Alpha: 0.1 - R²: 0.29258919311073694

**Conclusão**:
Todos os resultados são bem proximos e considerando as duas casas decimais, podemos dizer que sao iguais.


In [ ]:
#3. Regularização LASSO
r2_scores_lasso = []
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1] 

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    
    y_pred_lasso = lasso.predict(X_test)
    r2_lasso = r2_score(y_test, y_pred_lasso)
    r2_scores_lasso.append((alpha, r2_lasso))

for alpha, r2_lasso in r2_scores_lasso:
    print(f'LASSO - Alpha: {alpha}, R²: {r2}')

**Conclusão da Regularizaçao LASSO**:
- O R² é o mesmo para todos os alphas testados: 0.292
- O R² da Regressao LASSO é praticamente igual ao RIDGE

In [ ]:
#4. Modelo Stepwise
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out = 0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            print(included+[new_column])
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
selected_features = stepwise_selection(X_train, y_train, verbose=True)
reg_stepwise = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()

print(reg_stepwise.summary())


**Conclusões sobre o stepwise:**
- O modelo stepwise gerou um R² = 0.24, que é menor que a regularizacao Ridge e Lasso.

***5. Comparação de todos os modelos***:

In [ ]:
# Resultado do modelo Ridge
print(f"Coeficientes Ridge: {ridge.coef_}")
print(f"R² Ridge: {ridge.score(X_test, y_test)}")


In [ ]:
# Resultado do modelo Lasso
print(f"Coeficientes Lasso: {lasso.coef_}")
print(f"R² Lasso: {lasso.score(X_test, y_test)}")

In [ ]:
print(f"Coeficientes Stepwise: {reg_stepwise.params}")
print(f"R² Stepwise: {reg_stepwise.rsquared}")

In [ ]:
#6.Transformacao da renda em logaritmo para melhorar o R²
modelo = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'
md = smf.ols(modelo, data = df1)
reg_lasso = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1 
                        , alpha = 0)
reg_lasso.summary()

**Transformando a variavel 'renda' em logaritmo, obtivemos um aumento no R² para 0.357**

In [ ]:
#7. Criando e ajustando a árvore de regressão
tree_model = DecisionTreeRegressor(random_state=100)
tree_model.fit(X_train, y_train)

# Avaliando o modelo na base de teste
y_pred_tree = tree_model.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = tree_model.score(X_test, y_test)

print(f"R² Árvore de Regressão: {r2_tree}")

O R² da Arvore de Regressao foi maior que todos os modelos: 0.391